In [1]:
# Load necessary library and module
import warnings
warnings.filterwarnings('ignore')

import sys
import pandas as pd
import seaborn as sns
import numpy as np
import re
import string
import unicodedata
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from textblob import TextBlob, Word

[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /home/gitpod/nltk_data...


In [2]:
# Apply text pre-processing to DataFrame
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
snowball = nltk.stem.SnowballStemmer('english')
porter_stemmer = PorterStemmer()

from textblob import TextBlob, Word

In [6]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

In [13]:
dataset= pd.read_csv('dataset/DatasetPilpres2024_Twitter_10112023.csv', encoding='ISO-8859-1')

In [15]:
samples=(dataset[:10]['text'])

In [7]:
# stemming(sentence)
print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


In [8]:
def stemming(text):
    text = stemmer.stem(text)
    return text

In [17]:
print(samples)

0    RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??P...
1    RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??S...
2    RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??B...
3    RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??T...
4    #Politik #MuhaiminIskandar #cakimin #mahasiswa...
5    RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??J...
6    Bacapres Anies Baswedan belum menunjuk Ketua T...
7    HANYA GANJAR YANG KONSISTEN TOLAK ISRAEL DATAN...
8    Nakama ga matten da! Jama su na! ??? [SEBUAH U...
9    Bertemu Din Syamsuddin, Cak Imin Klaim Menang ...
Name: text, dtype: object


In [20]:
samples.describe()

count                                                    10
unique                                                   10
top       RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??P...
freq                                                      1
Name: text, dtype: object

In [27]:
print(stemming(samples[:1]))

AttributeError: 'Series' object has no attribute 'lower'

In [18]:
samples['clean_content'] = samples['text'].progress_apply(lambda x: stemming(x))

KeyError: 'text'

In [9]:
print(stemming(sentence))

ekonomi indonesia sedang dalam tumbuh yang bangga


In [ ]:
def text_preprocessing(text):
  text = text.lower()                                   # Lowercase all sentences
      #   text = text.strip()                                   # Remove whitespace
  text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
  text = re.sub(r"pic.twitter.com\S+", ' ', text)       # Remove custom URLs for twitter
  text = re.sub(r'\@([\w]+)',' ', text)                 # Remove Mention @
  text = re.sub(r'\#([\w]+)',' ', text)                 # Remove #TAGAR
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
  text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
      #   text = re.sub(r'\b\w{1,3}\b','',text)                 #Remove n-chars,Remove less than 3 chars, minimum 4 character allowed "\b[a-zA-Z0-9]{3}\b"
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Tahap-5: simbol
      #   text = re.sub(r'[0-9]+','', text)                     # Tahap-6: angka
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
  text = re.sub(' +',' ', text)                         #remove multiple whitespace
  text = re.sub(r'^[ ]|[ ]$','', text)                  # Tahap-9: spasi di awal dan akhir kalimat

  # text = re.sub('\b[a-zA-Z0-9]{3}\b','',text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii character
  word_tokens = word_tokenize(text) # Word tokenize
  

  # Define Indonesian stopwords removal
  stop_words = stopwords.words('english')  # NLTK Indonesian stopwords
  clean_words = [word for word in word_tokens if word not in stop_words] # stopwords removal
  clean_words = ' '.join(clean_words)

  words = set(nltk.corpus.words.words())
  word_cleaner = word_tokenize(clean_words)
  cleaner_words = [w for w in word_cleaner if w.lower() in words or not w.isalpha()] #remove non english
  cleaner = ' '.join(cleaner_words)

  #stemming with snowball update 07072022 stemmer terakhir
#   stem_token=word_tokenize(cleaner)
#   stem_words = [snowball.stem(w) for w in stem_token]
#   stem_clean= ' '.join(stem_words)

  #update13072022 pake lemmatization TextBlob
  tblob=TextBlob(clean_words)
  lemma= [Word(word).lemmatize("v") for word in tblob.words]
  lemma_clean= ' '.join(lemma)

  #   cleaner = " ".join(w for w in nltk.wordpunct_tokenize(clean_words) \
        #   if w.lower() in words or not w.isalpha())

  return lemma_clean